In [1]:
from quantum_mcp.preprocess import MolecularHamiltonian

In [4]:
xyz_fpath = 'h2.xyz'
molecular_data = MolecularHamiltonian(xyz_fpath, 'sto3g')

molecular_data.run_ccsd()
molecular_data.compute_obi_and_tbi()

overwrite output file: h2-pyscf.log
[QC_MCP] == Running HF ==
[QC_MCP] Total number of orbitals = 2
[QC_MCP] Total number of electrons = (1, 1)
[QC_MCP] E[HF] = -1.116325564486115
[QC_MCP] == Running R-CCSD ==
[QC_MCP] E[R-CCSD] = -1.1371758844013344
[QC_MCP] Computing the one- and two-electron integrals
[QC_MCP] Using one- and two-body integrals from HF in the MO basis
[QC_MCP] Generated the restricted molecular spin Hamiltonian
0.7080240981000804
